## Lab2 Assignment
> 1. Calculate and display the count of people (gender wise) who have taken treatment for mental health condition.
> 2. What is the average age of participants (gender wise) from the country "Canada"?
> 3. What is the youngest age of participant from United Kingdom?
> 4. List the count of participants who work for tech company , country wise?
> 5. List the top 10 participants by age who had family history of mental health issues and had taken the treament.

In [5]:
# Creating Spark Session
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [6]:
# Reading data
surveyDf = spark.read.options(header="true",\
                              inferSchema="true",\
                              nullValue="NA",\
                              timestampFormat="yyyy-MM-dd'T'HH:mm:ss",\
                              mode="failfast")\
                             .csv("/Users/sravanthi/Documents/Sandbox/spark/survey.csv")

In [7]:
#Summary of dataframe
surveyDf.summary()

DataFrame[summary: string, Timestamp: string, Age: string, Gender: string, Country: string, state: string, self_employed: string, family_history: string, treatment: string, work_interfere: string, no_employees: string, remote_work: string, tech_company: string, benefits: string, care_options: string, wellness_program: string, seek_help: string, anonymity: string, leave: string, mental_health_consequence: string, phys_health_consequence: string, coworkers: string, supervisor: string, mental_health_interview: string, phys_health_interview: string, mental_vs_physical: string, obs_consequence: string, comments: string]

In [8]:
# As there are many wrongly imputed data in gender column. categorized gender to 3 groups Female, Male, No gender

from pyspark.sql import functions as F
from pyspark.sql.functions import when   

surveyDf = surveyDf.withColumn("Gender",
                                F.when(surveyDf["Gender"] == "Cis Female", "Female")
                                .when(surveyDf["Gender"] == "cis-female/femme", "Female")
                                .when(surveyDf["Gender"] == "Woman", "Female")
                                .when(surveyDf["Gender"] == "F", "Female")
                                .when(surveyDf["Gender"] == "Femake", "Female")
                                .when(surveyDf["Gender"] == "femail", "Female")
                                .when(surveyDf["Gender"] == "Female (cis)", "Female")
                                .when(surveyDf["Gender"] == "female", "Female")
                                .when(surveyDf["Gender"] == "Cis Male", "Male")
                                .when(surveyDf["Gender"] == "Cis Man", "Male")
                                .when(surveyDf["Gender"] == "Guy (-ish) ^_^", "Male")
                                .when(surveyDf["Gender"] == "M", "Male")
                                .when(surveyDf["Gender"] == "Mail", "Male")
                                .when(surveyDf["Gender"] == "maile", "Male")
                                .when(surveyDf["Gender"] == "Make", "Male")
                                .when(surveyDf["Gender"] == "Mal", "Male")
                                .when(surveyDf["Gender"] == "Male (CIS)", "Male")
                                .when(surveyDf["Gender"] == "Male-ish", "Male")
                                .when(surveyDf["Gender"] == "Malr", "Male")
                                .when(surveyDf["Gender"] == "Man", "Male")
                                .when(surveyDf["Gender"] == "msle", "Male")
                                .when(surveyDf["Gender"] == "Male", "Male")
                                .when(surveyDf["Gender"] == "male", "Male")
                                .otherwise("No gender")
                                
                                )
          

In [9]:
# 1. Calculate and display the count of people (gender wise) who have taken treatment for mental health condition.

# Register the DataFrame as a SQL temporary view
surveyDf.createOrReplaceTempView("surveydf")
# spark.sql("select unique(gender) from surveydf")
sqlsurveyDf = spark.sql('select gender,count(gender) from surveyDf where mental_health_consequence == "Yes" and treatment = "Yes" group by gender ')
sqlsurveyDf.show()
                        

+---------+-------------+
|   gender|count(gender)|
+---------+-------------+
|No gender|           35|
|   Female|           24|
|     Male|          115|
+---------+-------------+



In [10]:
# 2. What is the average age of participants (gender wise) from the country "Canada"?

sqlsurveyDf1 = spark.sql('select gender, Avg(Age) as avg_age from surveyDf where Country == "Canada" group by gender ')
sqlsurveyDf1.show()


+---------+------------------+
|   gender|           avg_age|
+---------+------------------+
|No gender|             30.25|
|   Female|              28.6|
|     Male|29.181818181818183|
+---------+------------------+



In [11]:
# 3. What is the youngest age of participant from United Kingdom?

sqlsurveyDf2 = spark.sql('select Country,Age from surveyDf where Country == "United Kingdom" and Age > 0 order by Age limit 1 ')
sqlsurveyDf2.show()

+--------------+---+
|       Country|Age|
+--------------+---+
|United Kingdom| 18|
+--------------+---+



In [12]:
# 4. List the count of participants who work for tech company , country wise?

sqlsurveyDf3 = spark.sql('select country, count(*) as Count_participants  from surveyDf where tech_company == "Yes" group by country')
sqlsurveyDf3.show()

+-------------+------------------+
|      country|Count_participants|
+-------------+------------------+
|       Russia|                 3|
|       Sweden|                 7|
|  Philippines|                 1|
|    Singapore|                 4|
|      Germany|                40|
|       France|                12|
|       Greece|                 2|
|      Belgium|                 3|
|      Finland|                 2|
|United States|               611|
|        India|                 9|
|        China|                 1|
|      Croatia|                 2|
|      Nigeria|                 1|
|        Italy|                 5|
|       Norway|                 1|
|        Spain|                 1|
|      Denmark|                 2|
|      Ireland|                26|
|     Thailand|                 1|
+-------------+------------------+
only showing top 20 rows



In [13]:
# 5. List the top 10 participants by age who had family history of mental health issues and had taken the treament.
 
sqlsurveyDf3 = spark.sql('select age from surveyDf where family_history == "Yes" and mental_health_consequence == "Yes" and treatment = "Yes" order by age ')
sqlsurveyDf3.show()


+---+
|age|
+---+
| -1|
|  8|
| 21|
| 22|
| 22|
| 23|
| 23|
| 24|
| 24|
| 24|
| 24|
| 25|
| 25|
| 25|
| 25|
| 25|
| 25|
| 25|
| 25|
| 26|
+---+
only showing top 20 rows

